In [2]:


import pandas as pd

import report_util
import tables

heritability = tables.create_heritability_table('../data/heritability.csv')

In [3]:
df = pd.read_csv('../data/heritability.csv')
df.columns = df.columns.map(lambda x: x.replace('_', ' '))
rows = []
level = 0.05 / 3
for subject in df['subject'].unique():
    selected = df[df['subject'] == subject]
    for operator in df['crossover operator'].unique():
        selected2 = selected[selected['crossover operator'] == operator]
        selected1 = selected[selected['crossover operator'] == 'Linked']
        for metric, name, f in [('inheritance rate', 'IR', report_util.mann_whitney),
                                ('hybrid', 'HY', report_util.fisher_exact)]:
            values2 = selected2[metric]
            values1 = selected1[metric]
            p = f(values1, values2)
            rows.append([operator, subject, name, 'color: red;' if p < level else ''])
sigs = pd.DataFrame(rows, columns=['Crossover Operator', 'Subject', 'Metric', 'Value']) \
    .pivot(index=['Subject'], values=['Value'], columns=['Crossover Operator', 'Metric']) \
    .reorder_levels(axis=1, order=['Crossover Operator', 'Metric', None]) \
    .droplevel(2, axis=1)
display(sigs)

Crossover Operator    Two Point                 One Point              Linked  \
Metric                       IR           HY           IR           HY     IR   
Subject                                                                         
Ant                 color: red;  color: red;  color: red;  color: red;          
Bcel                color: red;  color: red;  color: red;  color: red;          
Closure             color: red;               color: red;  color: red;          
Maven               color: red;               color: red;                       
Nashorn             color: red;               color: red;  color: red;          
Rhino               color: red;               color: red;                       
Tomcat              color: red;               color: red;                       

Crossover Operator     
Metric             HY  
Subject                
Ant                    
Bcel                   
Closure                
Maven                  
Nashorn                
Rhino                  
Tomcat

In [4]:
styler = tables.style_table(heritability, axis=0) \
    .apply(lambda _: sigs, axis=None)
display(styler)
print(styler.to_latex(multicol_align='c', hrules=True, multirow_align='t', convert_css=True))

\begin{tabular}{lrrrrrr}
\toprule
 & \multicolumn{2}{c}{Linked} & \multicolumn{2}{c}{One Point} & \multicolumn{2}{c}{Two Point} \\
 & HY & IR & HY & IR & HY & IR \\
Subject &  &  &  &  &  &  \\
\midrule
Ant & 0.561 & {\cellcolor{yellow}} 0.923 & \color{red} 0.459 & \color{red} 0.124 & \color{red} 0.493 & \color{red} 0.069 \\
Bcel & 0.283 & 0.512 & \color{red} 0.660 & \color{red} 0.347 & {\cellcolor{yellow}} \color{red} 0.756 & \color{red} 0.286 \\
Closure & {\cellcolor{yellow}} 0.742 & 0.717 & {\cellcolor{yellow}} \color{red} 0.661 & \color{red} 0.101 & 0.712 & \color{red} 0.094 \\
Maven & 0.446 & 0.589 & 0.404 & {\cellcolor{yellow}} \color{red} 0.497 & 0.399 & {\cellcolor{yellow}} \color{red} 0.453 \\
Nashorn & 0.622 & 0.646 & \color{red} 0.548 & \color{red} 0.117 & 0.591 & \color{red} 0.132 \\
Rhino & 0.611 & 0.502 & 0.599 & \color{red} 0.263 & 0.643 & \color{red} 0.255 \\
Tomcat & 0.322 & 0.775 & 0.350 & \color{red} 0.276 & 0.328 & \color{red} 0.279 \\
\bottomrule
\end{tabular}

